In [3]:
from flair.models.text_classification_model import TARSClassifier
from flair.data import Sentence
from flair.data  import FlairDataset
from flair.datasets import SentenceDataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from util import *
import util
from flair.data import Corpus
from flair.trainers import ModelTrainer

In [4]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")
subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))

In [5]:
training_labels = training_set[subset].apply(util.array_labels, axis=1)
testing_labels = testing_set[subset].apply(util.array_labels, axis=1)
training_set['labels'] = training_labels
testing_set['labels'] = testing_labels
        

In [6]:
import funcy as f
#@f.collecting
def mk_sentence(row):
    sentence = Sentence(row.claims)
    for label in row.labels:
        sentence.add_label('tag', label)
    return sentence
        

def mk_dataset(df):
    sentences = df[['claims','labels']].apply(mk_sentence, axis=1)
    return SentenceDataset(list(sentences))

In [7]:
training_dataset = mk_dataset(training_set)

In [8]:
testing_dataset = mk_dataset(testing_set)

In [13]:
TARSClassifier?

In [11]:
corpus = Corpus(train=training_dataset, test=testing_dataset)

In [ ]:
from flair.trainers import ModelTrainer

In [ ]:
nice_subset = [tier_translations[x] for x in subset]

In [14]:
tars = TARSClassifier(task_name='tag', batch_size=8, label_dictionary=corpus.make_label_dictionary(), multi_label=True)

2020-12-07 16:20:17,179 Computing label dictionary. Progress:


100%|██████████| 1118/1118 [00:00<00:00, 2844.90it/s]

2020-12-07 16:20:17,576 [b'Analysis and Modeling', b'Analysis and Modeling: 3D Modeling', b'Anatomical Target', b'Anatomical Target: Lower Extremity', b'Anatomical Target: Lower Extremity - Hip', b'Anatomical Target: Lower Extremity - Knee', b'Anatomical Target: Torso', b'Anatomical Target: Torso - Spine', b'Anatomical Target: Upper Extremity', b'Anatomical Target: Upper Extremity - Shoulder', b'Imaging', b'Imaging: CT', b'Imaging: MRI', b'Manufacturing', b'Personalized Product: Guide or Jig', b'Specification of Use', b'Specification of Use: Joint Replacement', b'Personalized Product: Implant', b'Surgical Method', b'Manufacturing: Additive Manufacturing', b'Imaging: Ultrasound', b'Specification of Use: Disease']


In [15]:
trainer = ModelTrainer(tars, corpus)

In [17]:
trainer.train(base_path='models/tars/v1', # path to store the model artifacts
              learning_rate=0.02, # use very small learning rate
              mini_batch_size=2,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=10, # terminate after 10 epochs
              )

2020-12-07 16:20:47,940 ----------------------------------------------------------------------------------------------------
2020-12-07 16:20:47,943 Model: "TARSClassifier(
  (document_embeddings): None
  (decoder): None
  (loss_function): None
  (tars_model): TextClassifier(
    (document_embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_

2020-12-07 16:20:47,944 ----------------------------------------------------------------------------------------------------
2020-12-07 16:20:47,945 Corpus: "Corpus: 875 train + 97 dev + 243 test sentences"
2020-12-07 16:20:47,945 ----------------------------------------------------------------------------------------------------
2020-12-07 16:20:47,946 Parameters:
2020-12-07 16:20:47,947  - learning_rate: "0.02"
2020-12-07 16:20:47,947  - mini_batch_size: "2"
2020-12-07 16:20:47,948  - patience: "3"
2020-12-07 16:20:47,948  - anneal_factor: "0.5"
2020-12-07 16:20:47,949  - max_epochs: "10"
2020-12-07 16:20:47,950  - shuffle: "True"
2020-12-07 16:20:47,950  - train_with_dev: "False"
2020-12-07 16:20:47,951  - batch_growth_annealing: "False"
2020-12-07 16:20:47,951 ----------------------------------------------------------------------------------------------------
2020-12-07 16:20:47,952 Model training base path: "models/tars/v1"
2020-12-07 16:20:47,953 ---------------------------------

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 10.91 GiB total capacity; 9.54 GiB already allocated; 59.44 MiB free; 9.59 GiB reserved in total by PyTorch)